In [ ]:
from pyarrow.parquet import ParquetFile
import duckdb
import pandas as pd
%load_ext autoreload
%autoreload 2

%reload_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

conn = duckdb.connect()
%sql conn --alias duckdb

### Raw Pre Processing

In [ ]:
# Clinical Trials Data:
# https://clinicaltrials.gov/search -- download all as JSON, name as ctg-studies.json

In [ ]:
%%sql
copy
    (select * from read_json("ctg-studies.json", format = 'array', sample_size = -1))
    to 'ctg-studies.parquet'
    (format 'parquet', codec 'zstd');


In [ ]:
print(
    ParquetFile("ctg-studies.parquet").schema_arrow.to_string(
        show_schema_metadata=False, show_field_metadata=False
    )
)

In [ ]:
%%sql
raw_df <<
with refs as (
    select
        protocolSection.identificationModule.nctId as nctId,
        unnest(protocolSection.referencesModule.references) as references
    from 'ctg-studies.parquet'
        where protocolSection.designModule.studyType = 'INTERVENTIONAL'
)
select 
    protocolSection.identificationModule.nctId as nctId,
    protocolSection.statusModule.overallStatus as overallStatus,
    protocolSection.statusModule.lastKnownStatus as lastKnownStatus,
    protocolSection.statusModule.whyStopped as whyStopped,
    protocolSection.statusModule.startDateStruct as startDateStruct,
    protocolSection.statusModule.completionDateStruct as completionDateStruct,
    protocolSection.statusModule.primaryCompletionDateStruct as primaryCompletionDateStruct,
    protocolSection.designModule.studyType as studyType,
    protocolSection.designModule.phases as phases,
    protocolSection.outcomesModule as outcomes,
    protocolSection.referencesModule.seeAlsoLinks as seeAlsoLinks,
    protocolSection.descriptionModule.briefSummary as briefSummary,
    protocolSection.descriptionModule.detailedDescription as detailedDescription,
    protocolSection.referencesModule.availIpds as availIpds,
    hasResults as hasResults,
    r.references as reference
from 'ctg-studies.parquet'
    left join refs r on (r.nctId = protocolSection.identificationModule.nctId)
    where protocolSection.designModule.studyType = 'INTERVENTIONAL'

In [ ]:
raw_df.to_parquet("ctg-projected-raw.parquet")

### CTG Data

In [ ]:
raw_df = pd.read_parquet("ctg-projected-raw.parquet")

raw_df_proc = raw_df.join(
    pd.json_normalize(raw_df["startDateStruct"]).rename(columns=lambda c: f"start_{c}")
).drop(columns=["startDateStruct"])

raw_df_proc = raw_df_proc.join(
    pd.json_normalize(raw_df_proc["completionDateStruct"]).rename(
        columns=lambda c: f"completion_{c}"
    )
).drop(columns=["completionDateStruct"])

raw_df_proc = raw_df_proc.join(
    pd.json_normalize(raw_df_proc["primaryCompletionDateStruct"]).rename(
        columns=lambda c: f"primary_completion_{c}"
    )
).drop(columns=["primaryCompletionDateStruct"])

raw_df_proc = raw_df_proc.join(
    pd.json_normalize(raw_df_proc["reference"]).rename(
        columns=lambda c: f"reference_{c}"
    )
).drop(columns=["reference"])

raw_df_proc["start_date"] = pd.to_datetime(raw_df_proc["start_date"], format="mixed")

raw_df_proc["completion_date"] = pd.to_datetime(
    raw_df_proc["completion_date"], format="mixed"
)

raw_df_proc["primary_completion_date"] = pd.to_datetime(
    raw_df_proc["primary_completion_date"], format="mixed"
)

# Concat multiple phases together, if exists
raw_df_proc["phases"] = raw_df_proc["phases"].apply(
    lambda x: "-".join(y for y in x) if x is not None else x
)

# Includes only background publications
background_mask = raw_df_proc["reference_type"] == "BACKGROUND"

cols_to_keep = [
    "nctId",
    "overallStatus",
    "phases",
    "briefSummary",
    "detailedDescription",
    "hasResults",
    "start_date",
    "start_type",
    "completion_date",
    "completion_type",
    "primary_completion_date",
    "primary_completion_type",
    "reference_citation",
    "reference_pmid",
    "reference_type",
]

raw_df_proc = raw_df_proc[background_mask][cols_to_keep].reset_index(drop=True)

In [ ]:
raw_df_proc.head(3)

In [ ]:
raw_df_proc.to_parquet("ctg-projected-processed.parquet")